#### Declarative with Imperative Table (a.k.a. Hybrid Declarative)

`Declarative mappings` may also be provided with a _pre-existing_ `Table` object, or otherwise a `Table` or other arbitrary `FromClause` construct (such as a `Join` or `Subquery`) that is __constructed separately__.

This is referred to as a __`"hybrid declarative" mapping`__, as the class is mapped using the _declarative style_ for everything involving the `mapper configuration`, however the _mapped_ `Table` _object_ is __produced separately__ and _passed_ to the `declarative process` __directly__.

In [7]:
from sqlalchemy import Column, ForeignKey, Integer, String, Table, func, select
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()

In [3]:
# construct a Table directly
# the Base.metadata collection is usually a good choice
# for MetaData but any MetaData collection may be used
user_table = Table(
    "user",
    Base.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
    Column("fullname", String),
    Column("nickname", String),
)

In [4]:
# construct the User class using this table.
class User(Base):
    __table__ = user_table

Above, a `Table` object is constructed using the approach described at `Describing Databases with MetaData`. It __can then be applied directly__ to a class that is `declaratively mapped`. The `__tablename__` and `__table_args__` _declarative class attributes_ are __not used__ in this form. The above _configuration_ is __often more readable__ as an _inline definition_.

In [5]:
class UserInline(Base):
    __table__ = Table(
        "user_inline",
        Base.metadata,
        Column("id", Integer, primary_key=True),
        Column("name", String),
        Column("fullname", String),
        Column("nickname", String),
    )

A _natural effect_ of the above style is that the `__table__` attribute is itself __defined within the `class definition block`__. As such it __may be immediately referred__ towards _within subsequent attributes_, such as the example below which illustrates _referring to the type column_ in a __polymorphic__ `mapper configuration`.

In [6]:
class Person(Base):
    __table__ = Table(
        "person",
        Base.metadata,
        Column("id", Integer, primary_key=True),
        Column("name", String(50)),
        Column("type", String(50)),
    )
    
    __mapper_args__ = {
        "polymorphic_on": __table__.c.type,
        "polymorphic_identity": "person",
    }

The __`"imperative table"`__ form is also used when a __non-`Table` construct__, such as a `Join` or `Subquery` object, is _to be mapped_. An example below.

```
subq = (
    select(
        func.count(orders.c.id).label("order_count"),
        func.max(orders.c.price).label("highest_order"),
        orders.c.customer_id,
    ).group_by(orders.c.customer_id).subquery()
)

customer_select = (
    select(customers, subq).
    join_from(customers, subq, customers.c.id == subq.c.customer_id).
    subquery()
)


class Customer(Base):
    __table__ = customer_select
```

For _background_ on `mapping to non-Table constructs` see the sections `Mapping a Class against Multiple Tables` and `Mapping a Class against Arbitrary Subqueries`.

The `"imperative table"` form is of particular use __when the class itself is using an alternative form of attribute declaration__, such as `Python dataclasses`. See the section `Applying ORM Mappings to an existing dataclass` for detail.